<a href="https://colab.research.google.com/github/pratyush021/Image_Recognition_project/blob/master/face_data_git(working).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/shamaunalam/face_data.git

In [0]:
import matplotlib.pyplot as plt
import cv2
import numpy as np 
import pickle
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense,Dropout,Flatten
from keras.optimizers import Adam, RMSprop
from keras.utils.np_utils import to_categorical


In [0]:
with open('face_data/images.p','rb') as f:
  images = pickle.load(f)

with open('face_data/labels.p','rb') as f:
  labels = pickle.load(f)


In [0]:
plt.imshow(images[10],cmap='gray')
plt.title(labels[10],color = 'b')
plt.show()

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(images,labels,test_size=0.2)

In [0]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [0]:
num_of_samples = []

cols = 5
num_classes = len(set(labels))

fig, axs = plt.subplots(nrows = num_classes,ncols=cols,figsize=(10,25))
fig.tight_layout()

for i in range(cols):
  for j in range(num_classes):
    x_selected = x_train[y_train == j]
    axs[j][i].imshow(x_selected[np.random.randint(0,(len(x_selected)-1)),:,:],
                      cmap = plt.get_cmap('gray'))
    axs[j][i].axis('off')
    if i == 2:
      axs[j][i].set_title(le.inverse_transform(np.array([j])))
      num_of_samples.append(len(x_selected))

In [0]:
le.inverse_transform(np.array([1]))

nodes in the network<br>
32 32 64 64 128 128 510

In [0]:
def preprocess(img):
  img = cv2.equalizeHist(img)#Brightness equalization
  img = img.reshape(200,200,1)
  img = img/255
  return img  

In [0]:
x_train = np.array(list(map(preprocess,x_train)))#map applys preprocess all images in x_train and returns it as an object
x_test = np.array(list(map(preprocess,x_test)))

In [0]:
print(x_train.shape)
print(x_test.shape)

In [0]:
y_train = to_categorical(y_train,num_classes)
y_test = to_categorical(y_test,num_classes)

In [0]:
model = Sequential()

model.add(Conv2D(16,(5,5),input_shape=(200,200,1),activation='relu'))
model.add(Conv2D(16,(5,5),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3),activation='relu'))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(512,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))

model.compile(RMSprop(lr=0.01),loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
print(model.summary())

In [0]:
h = model.fit(x_train,y_train,epochs=20,verbose = 1,validation_data=(x_test,y_test))

In [0]:
model.save("Face_Recog.h5")